In [ ]:
!pip install numpy
!pip install pandas
!pip install fastai

from fastai.basics import *
from fastai.callback.all import *
from fastai.vision.all import *
import numpy
import pandas

In [ ]:
path = Path('/kaggle/input/sample/sample')
path.ls()

(#3) [Path('/kaggle/input/sample/sample/sample_labels.csv'),Path('/kaggle/input/sample/sample/images'),Path('/kaggle/input/sample/sample/sample')]

In [ ]:
df = pd.read_csv(path/f"sample_labels.csv")
df.columns

Index(['Image Index', 'Finding Labels', 'Follow-up #', 'Patient ID',
       'Patient Age', 'Patient Gender', 'View Position', 'OriginalImageWidth',
       'OriginalImageHeight', 'OriginalImagePixelSpacing_x',
       'OriginalImagePixelSpacing_y'],
      dtype='object')

In [ ]:
dblock = DataBlock()
dsets = dblock.datasets(df)

In [ ]:
from fastai.vision.all import *

def get_x(r):
    return path/'images'/r['Image Index']

def get_y(r):
    return r['Finding Labels'].split('|')

dblock = DataBlock(
    blocks=(ImageBlock, MultiCategoryBlock),
    get_x=get_x,
    get_y=get_y,
    splitter=RandomSplitter(),
    batch_tfms=[*aug_transforms(size=224), Normalize.from_stats(*imagenet_stats)]
)

dls = dblock.dataloaders(df, bs=64)

# Check the DataLoader outputs
images, labels = dls.one_batch()
print(f'Images shape: {images.shape}')  # Should be (64, 3, 224, 224)
print(f'Labels shape: {labels.shape}')  # Should be (64, num_classes)

Images shape: torch.Size([64, 3, 224, 224])
Labels shape: torch.Size([64, 15])


In [ ]:
import torch
import math
import torch.nn as nn
import torch.nn.functional as F

class BasicConv(nn.Module):
    def __init__(self, in_planes, out_planes, kernel_size, stride=1, padding=0, dilation=1, groups=1, relu=True, bn=True, bias=False):
        super(BasicConv, self).__init__()
        self.out_channels = out_planes
        self.conv = nn.Conv2d(in_planes, out_planes, kernel_size=kernel_size, stride=stride, padding=padding, dilation=dilation, groups=groups, bias=bias)
        self.bn = nn.BatchNorm2d(out_planes,eps=1e-5, momentum=0.01, affine=True) if bn else None
        self.relu = nn.ReLU() if relu else None

    def forward(self, x):
        x = self.conv(x)
        if self.bn is not None:
            x = self.bn(x)
        if self.relu is not None:
            x = self.relu(x)
        return x

class Flatten(nn.Module):
    def forward(self, x):
        return x.view(x.size(0), -1)

class ChannelGate(nn.Module):
    def __init__(self, gate_channels, reduction_ratio=16, pool_types=['avg', 'max']):
        super(ChannelGate, self).__init__()
        self.gate_channels = gate_channels
        self.mlp = nn.Sequential(
            Flatten(),
            nn.Linear(gate_channels, gate_channels // reduction_ratio),
            nn.ReLU(),
            nn.Linear(gate_channels // reduction_ratio, gate_channels)
            )
        self.pool_types = pool_types
    def forward(self, x):
        channel_att_sum = None
        for pool_type in self.pool_types:
            if pool_type=='avg':
                avg_pool = F.avg_pool2d( x, (x.size(2), x.size(3)), stride=(x.size(2), x.size(3)))
                channel_att_raw = self.mlp( avg_pool )
            elif pool_type=='max':
                max_pool = F.max_pool2d( x, (x.size(2), x.size(3)), stride=(x.size(2), x.size(3)))
                channel_att_raw = self.mlp( max_pool )
            elif pool_type=='lp':
                lp_pool = F.lp_pool2d( x, 2, (x.size(2), x.size(3)), stride=(x.size(2), x.size(3)))
                channel_att_raw = self.mlp( lp_pool )
            elif pool_type=='lse':
                # LSE pool only
                lse_pool = logsumexp_2d(x)
                channel_att_raw = self.mlp( lse_pool )

            if channel_att_sum is None:
                channel_att_sum = channel_att_raw
            else:
                channel_att_sum = channel_att_sum + channel_att_raw

        scale = F.sigmoid( channel_att_sum ).unsqueeze(2).unsqueeze(3).expand_as(x)
        return x * scale

def logsumexp_2d(tensor):
    tensor_flatten = tensor.view(tensor.size(0), tensor.size(1), -1)
    s, _ = torch.max(tensor_flatten, dim=2, keepdim=True)
    outputs = s + (tensor_flatten - s).exp().sum(dim=2, keepdim=True).log()
    return outputs

class ChannelPool(nn.Module):
    def forward(self, x):
        return torch.cat( (torch.max(x,1)[0].unsqueeze(1), torch.mean(x,1).unsqueeze(1)), dim=1 )

class SpatialGate(nn.Module):
    def __init__(self):
        super(SpatialGate, self).__init__()
        kernel_size = 7
        self.compress = ChannelPool()
        self.spatial = BasicConv(2, 1, kernel_size, stride=1, padding=(kernel_size-1) // 2, relu=False)
    def forward(self, x):
        x_compress = self.compress(x)
        x_out = self.spatial(x_compress)
        scale = F.sigmoid(x_out) # broadcasting
        return x * scale

class CBAM(nn.Module):
    def __init__(self, gate_channels, reduction_ratio=16, pool_types=['avg', 'max'], no_spatial=False):
        super(CBAM, self).__init__()
        self.ChannelGate = ChannelGate(gate_channels, reduction_ratio, pool_types)
        self.no_spatial=no_spatial
        if not no_spatial:
            self.SpatialGate = SpatialGate()
    def forward(self, x):
        x_out = self.ChannelGate(x)
        if not self.no_spatial:
            x_out = self.SpatialGate(x_out)
        return x_out

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class ConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels, cbam=False):
        super(ConvBlock, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=1, padding=1)
        self.bn = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.cbam = CBAM(out_channels) if cbam else nn.Identity()

    def forward(self, x):
        x = self.conv(x)
        x = self.bn(x)
        x = self.relu(x)
        x = self.cbam(x)
        return x

class CBAMNet(nn.Module):
    def __init__(self, num_classes):
        super(CBAMNet, self).__init__()
        self.layer1 = ConvBlock(3, 64, cbam=True)
        self.layer2 = nn.MaxPool2d(2)
        self.layer3 = ConvBlock(64, 128, cbam=True)
        self.layer4 = nn.MaxPool2d(2)
        self.layer5 = ConvBlock(128, 256, cbam=True)
        self.layer6 = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Linear(256, num_classes)

   A def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.layer5(x)
        x = self.layer6(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        return x

# Initialize the model
num_classes = len(dls.vocab)
model = CBAMNet(num_classes=num_classes)

In [ ]:
# Forward pass with a batch
model.eval()
with torch.no_grad():
    outputs = model(images)
print(f'Model output shape: {outputs.shape}')  # Should be (64, num_classes)


Model output shape: torch.Size([64, 15])


In [ ]:
from fastai.vision.all import *
import pandas as pd

# Create a Learner
learn = Learner(dls, model, loss_func=BCEWithLogitsLossFlat(), metrics=[accuracy_multi])

# Train the model
learn.fit_one_cycle(5, 1e-3)

epoch,train_loss,valid_loss,accuracy_multi,time


In [ ]:
# Check the data loader
batch = dls.one_batch()
images, labels = batch
print(images.shape)  # Should be (batch_size, 3, 224, 224)
print(labels.shape)  # Should be (batch_size, num_classes)

# Example: For a batch size of 64 and 14 classes, the shapes should be (64, 3, 224, 224) and (64, 14)
